In [1]:
import pandas as pd
df = pd.read_excel("data/law_清理過3.xlsx")

In [2]:
# 一個以上空白還沒處理
df['content'] = df['content'].str.replace(" ","")
df['content'] = df['content'].str.replace("一","1").str.replace("二","2").str.replace("三","3").str.replace("四","4").str.replace("五","5").str.replace("六","6").str.replace("七","7").str.replace("八","8").str.replace("九","9").str.replace("零","0").str.replace("○","0").str.replace("１","1").str.replace("５",'5').str.replace("０",'0')
df['content'] = df['content'].str.replace("千","").str.replace("十","").str.replace("\n","").str.replace("\u3000","").replace('及',"")

In [3]:
df2 = df[df['判決']=="准許"]
df2 = df2.reset_index(drop=True)

In [4]:
import re
regex_new = re.compile('([民同]法第1052條第\d(項|款)((第?\d+(?:、\d+)*款?)|以外之重大事由)?)')

In [5]:
law, law_id = [], []
for i,k in list(zip(df2['content'],df2['Id'])):
    law_temp = []
    match_new = re.findall(regex_new,i)
    for a in match_new:
        for b in a:
            if '1052' in b:
                law_temp.append(b)
    law.append(law_temp)
    law_id.append(k)
law1052 = list(zip(law, law_id))

In [6]:
law1052[0:3]

[(['民法第1052條第1項第3款', '民法第1052條第1項第3款', '民法第1052條第1項第3款'], 728),
 (['民法第1052條第2項', '民法第1052條第1項以外之重大事由', '民法第1052條第2項', '民法第1052條第2項'], 729),
 (['民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第1項以外之重大事由',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第2項'],
  730)]

### 最接近的字詞

找到兩者的position，在比較

In [7]:
judge_word = ["有理由","有據","即有理由","自屬有據","准許","判准"]

In [7]:
# [m.start() for m in re.finditer('民法第1052條第1項第3款', df2['content'][0])]

[1326, 2255, 5278]

In [38]:
# 最後一個字的位置
# df2['content'][0].rfind('判准')

3136

In [47]:
# a = [1326, 2255, 5278]
# b = 3136
# for i in a:
#     print(abs(i-b))

1810
881
2142


### Main

In [9]:
judge_law_list = []
for number in range(0,3156):
    for word in judge_word:
        try:
            if df2['content'][number].rfind(word) != -1:
                # 找出判決詞的位置
                index = df2['content'][number].rfind(word)
                break
        except:
            pass

    # 進行法條位置與判決詞距離位置的比對
    min_index = 10000000
    for law in set(law1052[number][0]):
        min_temp = []
        for i in [m.start() for m in re.finditer(law, df2['content'][number])]:
            min_temp.append(abs(i-index))
        # 比較每次的法條的位置，看有沒有更小的要更新
        if min(min_temp) < min_index:
            judge_law = law
            min_index = min(min_temp)
    judge_law_list.append(judge_law)

In [10]:
len(judge_law_list)

3156

In [11]:
df3 = pd.concat([df2, pd.DataFrame(judge_law_list, columns=['判決法條'])], axis=1)

In [13]:
df3.to_excel('data/law_判決法條測試3.xlsx')